In [232]:
import pandas as pd

In [233]:
data = pd.read_csv("player_regular_season.csv")

In [234]:
data['fullName'] = data['firstname'] + data['lastname']
data = data.drop(["firstname",'lastname', 'ilkid', 'firstname', 'lastname', 'team', 'leag'], axis=1)

In [235]:
player_years = {}
for index, row in data.iterrows():
    if row['fullName'] in player_years:
        player_years[row['fullName']].append(row['year'])
    else:
        player_years[row['fullName']] = [row['year']]
        

In [236]:
training_players = [player for player in player_years.keys() if len(player_years[player]) >= 5]
new_players = [player for player in player_years.keys() if (len(player_years[player]) == 1 and 2004 in player_years[player])]

In [237]:
data_index_full = data.set_index(['fullName'])

In [238]:
#separating the data set

training = data_index_full.loc[training_players]
test = data_index_full.loc[new_players]

In [239]:
training_data = []
for play in training_players:
    for index, row in data.iterrows():
        if row['fullName'] == play and row['year'] == player_years[play][0]:
            training_data.append(row)

training_data = pd.DataFrame(training_data).set_index(['fullName'])

In [243]:
training_data_clean = []
#get the mediam of players that played more than one team in the same year
for play in training_players:
    if len(training_data.loc[play]) != 18:
        aux = training_data.loc[play].median(numeric_only=False)
        aux.name = play
        training_data_clean.append(aux)
    else:
        training_data_clean.append(training_data.loc[play])
training_data_clean = pd.DataFrame(training_data_clean).reset_index()

In [244]:
#tag each player in the training data in a number
players_num = {}
for num, player in enumerate(training_players):
    players_num[player] = num
training_data_clean['play_num'] = players_num.values()


In [245]:
training_data_clean= training_data_clean.drop(['index'], axis=1)

In [246]:
training_data_clean = training_data_clean.dropna()
training_data_X = training_data_clean.drop(['play_num', 'year'], axis=1)
training_data_Y = training_data_clean['play_num']

In [247]:
test_X = test.reset_index().drop(['fullName', 'year'], axis=1)
test_X.head()

,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,77,1261,492,85,136,221,64,76,24,77.0,156,387,184,152,112,31,12
1,59,736,196,59,126,185,16,21,8,52.0,157,175,76,55,43,3,1
2,80,1382,468,89,153,242,85,70,18,73.0,150,389,172,174,121,13,3
3,30,384,109,47,71,118,12,12,24,12.0,87,78,45,40,19,1,0
4,2,12,5,0,3,3,0,0,0,0.0,1,5,2,0,0,2,1


In [248]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=len(training_data_Y), random_state=0).fit(training_data_X, training_data_Y)

In [249]:
pred =kmeans.predict(test_X)

In [257]:
def find_key_by_index(_dict, search_num):
    for player, num in _dict.items():
        if num == search_num:
            return player

In [267]:
for player_ass, player in zip(pred, test.index.values):
    print( ("%s ->> %s") %(player, find_key_by_index(players_num,player_ass)))
    print(data_index_full.loc[player])
    print(data_index_full.loc[find_key_by_index(players_num,player_ass)])
    break

Tony Allen ->> SamPellom
year        2004.0
gp            77.0
minutes     1261.0
pts          492.0
oreb          85.0
dreb         136.0
reb          221.0
asts          64.0
stl           76.0
blk           24.0
turnover      77.0
pf           156.0
fga          387.0
fgm          184.0
fta          152.0
ftm          112.0
tpa           31.0
tpm           12.0
Name: Tony Allen, dtype: float64
           year  gp  minutes  pts  oreb  dreb  reb  asts  stl  blk  turnover  \
fullName                                                                       
SamPellom  1979  44      373  109    28    64   92    18   12   12      18.0   
SamPellom  1980  77     1472  453   122   234  356    48   50   92      99.0   
SamPellom  1981  69     1037  289    90   139  229    28   29   47      57.0   
SamPellom  1982   2        9    4     0     0    0     1    0    0       0.0   
SamPellom  1982   6       29   12     2     6    8     1    0    0       2.0   
SamPellom  1982   4       20    8     2 

In [265]:
data.head()

,year,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm,fullName
0,1946,47,0,527,0,0,0,35,0,0,0.0,161,834,202,178,123,0,0,JohnAbramovic
1,1946,30,0,65,0,0,0,20,0,0,0.0,46,91,23,35,19,0,0,ChetAubuchon
2,1946,4,0,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,NormBaker
3,1946,58,0,138,0,0,0,16,0,0,0.0,98,263,53,69,32,0,0,HerschelBaltimore
4,1946,58,0,295,0,0,0,54,0,0,0.0,164,438,124,79,47,0,0,JohnBarr
